In [1]:
from Classes.Arquivo import Arquivo
from Classes.BancoDados import BancoDados

In [12]:
try:
    arquivo = Arquivo("Arquivos CSV/suicidios-total.csv")
    if arquivo.verifica:
        print("Arquivo CSV inspecionado com sucesso")
    else:
        print("O arquivo possui dados ausentes ou não possui dados")
except Exception as e:
    print(f"Erro ao inspecionar o arquivo: {e}")

Arquivo CSV inspecionado com sucesso
